In [1]:
import numpy as np
import string

np.random.seed(1234)

In [7]:
initial = {} # start of a phrase
first_order = {} # second word only
second_order = {}

In [3]:
def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

In [4]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘robert_frost.txt’ already there; not retrieving.



In [28]:
# def add2dict(d, k, v):
#     if k not in d:
#         d[k] = []
#     d[k].append(v)


def add2dict(d, k, v):
    if k not in d:
        d[k] = []
    if isinstance(d[k], list):
        d[k].append(v)
    else:
        d[k] = [d[k], v]
# [cat, cat, dog, dog, dog, dog, dog, mouse, ...]

In [29]:
for line in open('robert_frost.txt'):
    tokens = remove_punctuation(line.rstrip().lower()).split()

    T = len(tokens)
    for i in range(T):
        t = tokens[i]
        if i == 0:
            # measure the distribution of the first word
            initial[t] = initial.get(t, 0.) + 1
        else:
            t_1 = tokens[i-1]
            if i == T - 1:
                # measure probability of ending the line
                add2dict(second_order, (t_1, t), 'END')
            if i == 1:
                # measure distribution of second word
                # given only first word
                add2dict(first_order, t_1, t)
            else:
                t_2 = tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)

In [30]:
# normalize the distributions
initial_total = sum(initial.values())
for t, c in initial.items():
    initial[t] = c / initial_total

In [33]:
# convert  [cat, cat, cat, dog, dog, dog, dog, mouse, ...]
# into {cat: 0.5, dog: 0.4, mouse: 0.1}

# def list2pdict(ts):
#     # turn each list of possibilities into a dictionary of probabilities
#     d = {}
#     n = len(ts)
#     for t in ts:
#         d[t] = d.get(t, 0.) + 1
#     for t, c in d.items():
#         d[t] = c / n
#     return d



def list2pdict(ts):
    # turn each list of possibilities into a dictionary of probabilities
    d = {}
    n = len(ts)
    for t in ts:
        if isinstance(t, dict):
            t = tuple(t.items())
        d[t] = d.get(t, 0.) + 1
    for t, c in d.items():
        d[t] = c / n
    return d

In [34]:
for t_1 , ts in first_order.items():
    # replace list with dictionary of probabilities
    first_order[t_1] = list2pdict(ts)

In [35]:
for k, ts in second_order.items():
    second_order[k] = list2pdict(ts)

In [36]:
def sample_word(d):
    # print "d:", d
    p0 = np.random.random()
    # print "p0:", p0
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
    assert(False) # should never get here

In [37]:
def generate():
    for i in range(4): # generate 4 lines
        sentence = []

        # initial word
        w0 = sample_word(initial)
        sentence.append(w0)

        # sample second word
        w1 = sample_word(first_order[w0])
        sentence.append(w1)

        # second-order transitions until End
        while True:
            w2 = sample_word(second_order[(w0, w1)])
            if w2 == 'END':
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

In [41]:
generate()

the situation was like this the suns a wizard
of getting home again because
to watch his woods fill up with
to curl a little longer i mustnt feel too hurried


In [ ]:
# Exercise:
# 
# Determine the vocabulary size (V)
# We know that pi has shape V, A1 has shape V x V, and A2 has shape V x V x V
#
# In comparison, how many values are stored in our dictionaries?

In [ ]:
# Exercise 2:
# We can skip the step where we accumulate all the possible next words in a list 
# E.g. [cat, cat, dog, dog, dog, ...]
#
# Instead, like we do with the initial state distribution, creat the dictionary
# of counts directly as you loop through the data.
#
# You'll no longer need list2pdict()

In [43]:
# Define a list with various elements including a dictionary
ts = [1, 'hello', {'a': 10, 'b': 20}, 3.14, 1]

# Define a dictionary to store counts
d = {}

# Iterate over the list and handle dictionaries
for t in ts:
    if isinstance(t, dict):
        t = tuple(t.items())  # Convert dictionary to tuple of key-value pairs
    d[t] = d.get(t, 0) + 1

# Display the resulting dictionary with counts
print(d)

{1: 2, 'hello': 1, (('a', 10), ('b', 20)): 1, 3.14: 1}
